# ETF

In [8]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime

In [9]:
data = pd.read_csv('data_0514_20260119.csv',encoding='CP949',dtype=str)

In [10]:
start = "2025-06-01"
end = "2026-01-30"

Kospi_prices = yf.download('^KS11',start=start,end=end)['Close']['^KS11']

weekly_return1 = Kospi_prices.pct_change(periods=5)[5:] * 100 

weekly_return1.dropna(inplace=True)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [11]:
def RS_week(ticker):

    try:
        ETF_prices = yf.download(f'{ticker}.KS',start=start,end=end)['Close'][f'{ticker}.KS']
        
        weekly_return2 = ETF_prices.pct_change(periods=5)[5:] * 100

        weekly_return2.dropna(inplace=True)

        RS = weekly_return2 - weekly_return1

        return RS,ticker
    
    except:
        print(ticker)

In [12]:
corp_code = data['종목코드']
corp_list = []

for i in corp_code:
    st,ticker = RS_week(i)
    if st[-1]>=0:
        corp_list.append(ticker)

df = pd.DataFrame(corp_list,columns=['corp_list'])


# st = RS_week('298020')

# st.plot()

# plt.axhline(0, color='red', linestyle='--', linewidth=1);

# plt.show();

[*********************100%***********************]  1 of 1 completed
C:\Users\USER\AppData\Local\Temp\ipykernel_12152\1603132025.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if st[-1]>=0:
[*********************100%***********************]  1 of 1 completed
C:\Users\USER\AppData\Local\Temp\ipykernel_12152\1603132025.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if st[-1]>=0:
[*********************100%***********************]  1 of 1 completed
C:\Users\USER\AppData\Local\Temp\ipykernel_12152\1603132025.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, in

In [13]:
today_str = datetime.now().strftime('%Y-%m-%d')

file_name = f'corp_list_{today_str}.xlsx'

df.to_csv(file_name, index=False)
print(f"✅ {file_name} 파일 생성 및 저장 완료!")

✅ corp_list_2026-01-22.xlsx 파일 생성 및 저장 완료!


In [14]:
a= pd.read_csv('corp_list_2026-01-21.xlsx')
b= pd.read_csv('corp_list_2026-01-22.xlsx')

In [15]:
# '기업코드'라는 컬럼명을 사용한다고 가정합니다.
# 실제 컬럼명에 맞춰 수정하세요.
codes_a = set(a['corp_list'])
codes_b = set(b['corp_list'])

# 2. 겹치는 기업코드 (교집합)
intersect_codes = codes_a & codes_b
print(f"겹치는 코드 개수: {len(intersect_codes)}")

# 3-2. b에만 있는 코드 (b - a)
only_b = codes_b - intersect_codes

print(f"22일에만 있는 코드: {len(only_b)}")

겹치는 코드 개수: 157
22일에만 있는 코드: 79


In [16]:
intersect_df = pd.DataFrame(only_b, columns=['기업코드'])
intersect_df

,기업코드
0,105010
1,446690
2,442260
3,289040
4,473490
...,...
74,266370
75,091170
76,0150K0
77,414780


In [17]:
# import sys
# !{sys.executable} -m pip install -U finance-datareader

'''
숫자 코드,분류 명칭,설명
1,국내 주식형,"KOSPI 200, 삼성그룹주 등 국내 주식에 투자"
2,국내 채권형,"국고채, 회사채 등 국내 채권에 투자"
3,해외 주식형,"미국 나스닥100, 중국 CSI300 등 해외 주식에 투자"
4,해외 채권형,"미국채 10년, 글로벌 하이일드 등 해외 채권에 투자"
5,파생상품형,"레버리지, 인버스 등 선물/옵션을 활용한 상품"
6,원자재/통화,"금, 원유, 달러 선물 등에 투자"
7,부동산/기타,리츠(REITs)나 혼합 자산 상품
'''

In [36]:
import FinanceDataReader as fdr
import pandas as pd

# 2. KRX 전체 종목 리스트(주식, ETF 포함) 가져오기
# 처음 한 번만 실행하면 됩니다.
df_etfs = fdr.StockListing('ETF/KR')

# 3. 데이터 결합 (기업코드 기준)
# Symbol(종목코드)과 Name(종목명) 컬럼을 매칭합니다.
result = pd.merge(intersect_df, df_etfs[['Symbol', 'Name','Category']], 
                  left_on='기업코드', right_on='Symbol', how='left')

# print(df_etfs.columns)
# 4. 결과 출력
# print("--- 금융상품 이름 매칭 결과 ---")
# print(result[['기업코드', 'Name','Category']])

result = result[['기업코드', 'Name','Category']]
result.to_csv(f'matching_result_{today_str}.xlsx',index=False)